In [1]:
import torch
import numpy as np
import os
import torchvision
from voc import *
from coco import *
import torchvision.transforms as transforms
from torchvision.models import resnet152, resnet101, resnet18, resnet34, resnet50
from tqdm import tqdm
import json


/home/seongha/anaconda3/envs/MGSSL/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

avail_pretrained_models = timm.list_models(pretrained=True)
m_names = ['vit_base_patch16_224']


m_li = [resnet18(pretrained=True), resnet34(pretrained=True), resnet50(pretrained=True), resnet101(pretrained=True), resnet152(pretrained=True)]
model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=0)
model = timm.create_model('resnet10t', pretrained=True)
model.reset_classifier(0)

config = resolve_data_config({}, model=model)
transform = create_transform(**config)

m_li.append(model)

In [5]:
model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=0)
model.reset_classifier(0)

config = resolve_data_config({}, model=model)
transform = create_transform(**config)

In [10]:
def get_feat_extr_resnet(model):
    feat_extractors = []
    tmp = []
    for name, module in model.named_children():
        print(name)
        if name=='layer1' or name=='layer2' or name=='layer3' or name=='layer4' or name=='global_pool':
            feat_extractors.append(torch.nn.Sequential(*(tmp+[model.global_pool])))
            tmp.append(module)
        else:
            tmp.append(module)
    len(feat_extractors)
    return feat_extractors
feat_extractors=[]
for i in range(12):
    vit = [model.patch_embed,
    model.pos_drop,
    model.blocks[:i+1],
    model.norm,
    model.fc_norm,
    model.head]
    feat_extractors.append(torch.nn.Sequential(*vit))
print(feat_extractors)

AttributeError: 'ResNet' object has no attribute 'patch_embed'

In [4]:

# train_dataset = Voc2007Classification('data/voc', 'trainval', inp_name='data/voc/voc_glove_word2vec.pkl', LT=True)
train_dataset = COCO2014('data/coco', phase='train', inp_name='data/coco/coco_glove_word2vec.pkl', LT=True)
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])
train_dataset.transform = transforms.Compose([
                MultiScaleCrop(224, scales=(1.0, 0.875, 0.75, 0.66, 0.5), max_distort=2),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                normalize,
            ])
train_loader = torch.utils.data.DataLoader(train_dataset,
                                            batch_size=32, shuffle=False,
                                            num_workers=4)

[dataset] Done!
[annotation] Done!
[json] Done!


In [5]:
## get feat representation from pretrained layers
def get_Z(train_loader, g):
  for i, (input, target) in tqdm(enumerate(train_loader)):
    target[target == 0] = 1
    target[target == -1] = 0

    feat_Var = torch.autograd.Variable(input[0]).float().detach()
    if i==0:
      Z=g(feat_Var).detach().numpy()
      labels = target
    Z = np.concatenate((Z, g(feat_Var).detach().numpy()), axis=0)
    labels = np.concatenate((labels, target), axis=0)
  return Z, labels

In [6]:
## calculate transferability score
## Z: Feature from single forward pass of target sample (tensor)
def get_transferability(Z, labels, class_num=[], eps=1e-4,):
  n, d = Z.shape #1090, 1000
  
  # print(Z.shape, torch.t(Z).shape)
  inp = torch.eye(d) + torch.matmul(torch.t(Z), Z) * 1/(n*eps)
  first_term = 0.5 * torch.logdet(inp)
  print("first term: ", first_term)
  second_term = 0
  total_instance = 0
  for i, n_c in enumerate(class_num):
    Z_c = Z[np.where(labels[:,i] == 1)]
    # print(Z_c.shape[0])
    total_instance += Z_c.shape[0]
    second_term += n_c/(2*sum(class_num)) * torch.logdet(torch.eye(d) + 1/(n_c*eps) * torch.matmul(torch.t(Z_c), Z_c))
  print("second_term: ", second_term)
  # print(total_instance)

  return first_term - second_term

In [7]:
## get class num list
with open('./voc_class_num.json', 'r') as f:
  voc_class_num = json.load(f)
li = []
for k, v in voc_class_num.items():
  pos_cnt = v["1"]
  if "0" in v.keys():
    pos_cnt += v["0"]
  li.append(pos_cnt)

li = list(np.load('coco_class_num.npy'))
print(sum(li))


8155.0


In [13]:
for g in feat_extractors:
  Z, labels = get_Z(train_loader, g)
  Z = torch.as_tensor(Z)
  print(Z.shape)
  Z = torch.flatten(Z, start_dim=1)
  print(Z.shape)
  ## centralize Z
  Z = Z - torch.mean(Z)
  print(torch.mean(Z))

  ##scale down to trace
  Z = Z.reshape(Z.shape[0], Z.shape[1])
  Z /= torch.sqrt(torch.trace(torch.matmul(Z,torch.t(Z))))

  score = get_transferability(Z, labels, li)
  print(score)

34it [00:09,  3.66it/s]


torch.Size([1090, 196, 768])
torch.Size([1090, 150528])
tensor(-7.9398e-09)


In [11]:
for model in m_li[-1:]:
  feat_extractors = get_feat_extr_resnet(model)
  for g in feat_extractors:
    Z, labels = get_Z(train_loader, g)
    Z = torch.as_tensor(Z)
    print(Z.shape)
    Z = torch.flatten(Z, start_dim=1)
    print(Z.shape)
    ## centralize Z
    Z = Z - torch.mean(Z)
    print(torch.mean(Z))

    ##scale down to trace
    Z = Z.reshape(Z.shape[0], Z.shape[1])
    Z /= torch.sqrt(torch.trace(torch.matmul(Z,torch.t(Z))))

    score = get_transferability(Z, labels, li)
    print(score)

conv1
bn1
act1
maxpool
layer1
layer2
layer3
layer4
global_pool
fc


59it [00:07,  8.34it/s]


torch.Size([1893, 64])
torch.Size([1893, 64])
tensor(-3.6021e-08)
first term:  tensor(1.1719)
second_term:  tensor(1.1424)
tensor(0.0294)


59it [00:09,  6.21it/s]


torch.Size([1893, 64])
torch.Size([1893, 64])
tensor(-6.8012e-08)
first term:  tensor(1.1828)
second_term:  tensor(1.1533)
tensor(0.0296)


...

torch.Size([1893, 128])
torch.Size([1893, 128])
tensor(-5.1135e-08)
first term:  tensor(1.5084)
second_term:  tensor(1.4430)
tensor(0.0654)


59it [00:11,  5.04it/s]


torch.Size([1893, 256])
torch.Size([1893, 256])
tensor(-5.8566e-09)
first term:  tensor(1.8629)
second_term:  tensor(1.7569)
tensor(0.1059)


59it [00:18,  3.28it/s]


torch.Size([1893, 512])
torch.Size([1893, 512])
tensor(-1.7003e-08)
first term:  tensor(2.5729)
second_term:  tensor(2.5034)
tensor(0.0695)
